# Greenberger–Horne–Zeilinger (GHZ) State Fidelity

The aim of this benchmark is to show whether a GHZ state with high enough fidelity can be prepared such that the state is genuinely multipartite entangled. One can show that a fidelity larger than 0.5 is a sufficient condition (see Leibfried, D. et al., Creation of a six-atom ‘Schrödinger cat’ state. Nature 438, 639–642 (2005))

The benchmark currently offers two methods to estimate the fidelity:
* Multiple quantum coherences (G. J. Mooney et al., Generation and verification of 27-qubit Greenberger-Horne-Zeilinger states in a superconducting quantum computer, J. Phys. Commun. 5, 095004 (2021))
* Randomized measurements (Elben, A. et al., Statistical correlations between locally randomized measurements: A toolbox for probing entanglement in many-body quantum states, Phys. Rev. A 99, 052323 (2019))

Additionally, for a given set of $n$ qubits, different circuits can be applied which lead to the same GHZ state. Currently there are the following implementations:
* "naive": The textbook ciruit of depth $n$, which starts from a Hadamrd gate on one qubit and entangles each additional qubit with a CNOT operation from the first.
* "log_depth": A logarithmic depth circuit utilizing parallel CNOT applications. Either the method in Cruz et al. https://arxiv.org/abs/1807.05572 or the method in Mooney et al. https://arxiv.org/abs/2101.08946 is used, depending on which yields the lower depth or the lower number of 2-qubit gates. 
* "tree": An in-house logarithmic depth circuit utilizing calibration data of the CZ-fidelities and a tree-graph algorithm to find a circuit with minimal depth and high fidelity 2-qubit gates. 

Note that the first two methods will always give the same circuit for the same qubit layout (and the same coupling map), while the "tree" method relies on the latest calibration fidelities and can give different circuits on different days. 
Thus for consistent comparisons one of the first two methods should be chosen, while for the best results, i.e. the largest possible GME entangled GHZ state, the tree-method is preferable.

In [ ]:
from iqm.benchmarks.entanglement.ghz import *
import os
from iqm.qiskit_iqm import IQMProvider

token = "XXXXXXX"
os.environ["IQM_TOKEN"] = token
quantum_computer = "qc_name" # provide actual quantum computer name
iqm_server_url = "iqm_server_url" # provide actual IQM server URL
os.environ["IQM_SERVER_URL"] = iqm_server_url
provider = IQMProvider(iqm_server_url, quantum_computer=quantum_computer)
backend = provider.get_backend()

## Dynamical decoupling strategy

The success of dynamical decoupling (DD) depends on the coherence several factors, like
* Amount and lenght of idle segments in the circuit,
* Magnitude of idling errors,
* Single qubit gate fidelities (since DD adds additional single qubit gates).

Therefore just using the default DD strategy is not guaranteed to improve the results. In this notebook we apply a minimal DD strategy which only add single qubit gates on qubits with a high single qubit gate fidelity, and only adds a minimal amount of extra gates per idle segment (just tow X-Gates). This strategy should improve outcomes in most cases.

In [ ]:
from iqm.iqm_client.models import DDStrategy

strategy = DDStrategy(gate_sequences=[(2, 'XX', 'center')], target_qubits = None)

print(f"Minimal dynamical decoupling strategy:")
for k, v in strategy.__dict__.items():
    print(f"\t{k}: {v}")

## Definition of the benchmark configuration
The important parameters are:
* ``custom_qubits_array``: A ``list[list[int]]`` which includes all qubit layouts on which the benchmark is run.
* ``shots``: The number of shots for the fidelity measurement
* ``fidelity_routine``: Either "coherences" or "randomized_measurements"
* ``rem``: Boolean value that controls whether readout error mitigation is used
* ``mit_shots``: Whenever rem=True, this parameter controls the total number of shots used to calibrate readout error mitgation
* ``num_RMs``: The number of randomized measurement settings (only necessary when choosing ``fidelity_routine=randomized_measurements``)

In [ ]:
MINIMAL_GHZ = GHZConfiguration(
    state_generation_routine="tree",
            custom_qubits_array=[ # These are naive settings, replace with better qubit layouts!
                list(np.arange(10)),
                list(np.arange(20)),
            ],
    shots=1000,
    fidelity_routine="coherences", 
    rem=True,
    mit_shots=1000,
    use_dd = True,
    dd_strategy=strategy,
    quantum_computer=quantum_computer
)

### Improved qubit selection via layout fidelity graph

For thinking about which qubit layouts to use, the following visualization of the connectivity and CZ fidelities is helpful.

Requirements: 
* Access to the backend, in this example IQM Garnet.
* An access token environment variable needs to be set via ``os.environ["IQM_TOKEN"] = <your_token>``.

Use of the plot:
* If the ``qubit_layouts`` argument is provided, the selected qubits are marked in orange.
* CZ errors are indicated with edge width (**thinner edge is better**), where the edge width is given by $w_{ij} = - \mathrm{log}(\mathcal{F}_{\mathrm{CZ}}^{ij})$.
* Each edge is also labeled with the width value. 
* Some graph layouts are predefined to match the layout as shown in IQM-Resonance. If the layout is not predefined, a graph in grid or star layout will be automatically generated, dependeing on the backend. Automatically generated graphs might need to be rerun a few times until a nice node layout is found.
* Single qubit errors are visualized as node size (**smaller radius is better**), with the node size being determined by $- \mathrm{log}(\mathcal{F})$, where $\mathcal{F}$ is either the single qubit average gate fidelity, the single qubit readout fidelity, or the single qubit idle gate fidelity. Which of these is used can be controlled by the ``sq_metric`` argument, which can be set to "fidelity" "readout", or "coherence", where the latter option leads to the idle fidelities being shown.
* If the option `show_ghz_path` is set to True, the connections used for creating the GHZ state are highlighted red. Furthermore the "Edge map" is printed, which is the list of qubit pairs on which CZ gates act in the GHZ circuit. The list is ordered, so the first pair is scheduled first, and so on. CZ-Gates on subsequent pairs that don't share the same qubit are executed in parallel. 

In [ ]:
from iqm.benchmarks.utils_plots import plot_layout_fidelity_graph
import os

print(f"Check if access token environment variable is set: {'IQM_TOKEN' in os.environ}")
qubit_layouts = [list(range(20))]
fig = plot_layout_fidelity_graph(iqm_server_url, backend, quantum_computer = quantum_computer, qubit_layouts = qubit_layouts, sq_metric="coherence", show_ghz_path = True)

## Running the benchmark

In [ ]:
benchmark_ghz = GHZBenchmark(backend, MINIMAL_GHZ)
run_ghz = benchmark_ghz.run()

In [ ]:
result = benchmark_ghz.analyze()

## Accessing the results
To see individual fidelitiy and uncertainty values of a given qubit layout, one can filter the ``result.observations``-list by layout as shown below. 

The plot allows a comparison of all layout results with and without REM, where the data point description labels "L0", "L1", ... enumerate the layouts in the order defined in the configuration.

In [ ]:
qubit_layout = list(range(20))
for observation in result.observations:
    if observation.identifier.string_identifier == str(qubit_layout):
        print(f"{observation.name}: {observation.value} +/- {observation.uncertainty}")
result.plot_all()